<a href="https://colab.research.google.com/github/bingshengxuuu/bingshengxuuu.github.io/blob/main/Vincent_naive_bayes_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd

import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import numpy as np

import re

In [116]:
!wget https://raw.githubusercontent.com/EntropiaTsai/nccu_elt_course_material/refs/heads/main/fake_news.csv

--2025-03-09 06:31:53--  https://raw.githubusercontent.com/EntropiaTsai/nccu_elt_course_material/refs/heads/main/fake_news.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 726216 (709K) [text/plain]
Saving to: ‘fake_news.csv.3’

fake_news.csv.3     100%[===================>] 709.20K  --.-KB/s    in 0.06s   

2025-03-09 06:31:53 (12.4 MB/s) - ‘fake_news.csv.3’ saved [726216/726216]



In [117]:
# 使用 pandas 讀取 CSV 文件
df = pd.read_csv('fake_news.csv',encoding='utf-8')
# 顯示前五行數據
print(df.head(5))
len(df)

                                                text  label
0  星巴克Starbucks詐騙邀請LINE訊息. 原始謠傳版本：. 統一星巴克starbuck...      1
1  转摘：\n我给大家透漏个家传秘密，每天出门前用棉签蘸点小磨香油，滴进两个鼻孔内，轻捏几下即可...      1
2  各位朋友請注意，剛才在承德路被警察攔下臨檢，因為當時沒有戴口罩，被警察詢問車上有沒有乘客，我...      1
3  亡國危機迫在眉睫- -家庭觀念及性別認同危機。這支影片看後你我都有責仼，請及時踴躍發聲！轉發...      1
4  (shiny)臭跩貓回來囉(shiny)\n限時活動發放貼圖主題\n超可愛白爛貓愛嗆人嗆不停...      1


1000

In [118]:
repost = []
instant = []
fear = []
crowd = []
emoji = []

for text in df['text']:

    if re.search('轉[發載摘貼]*', text):
        result_repost = 1
    else:
        result_repost = 0
    repost.append(result_repost)

    if re.search('立[刻即馬]*', text):
        result_instant = 1
    else:
        result_instant = 0
    instant.append(result_instant)

    if re.search('可?怕', text):
        result_fear = 1
    else:
        result_fear = 0
    fear.append(result_fear)

    if re.search('各位', text):
        result_crowd = 1
    else:
        result_crowd = 0
    crowd.append(result_crowd)

    if re.search(r"[\U0001F600-\U0001F64F]", text):
        result_emoji = 1
    else:
        result_emoji = 0
    emoji.append(result_emoji)



df['repost'] = repost
df['instant'] = instant
df['fear'] = fear
df['crowd'] = crowd
df['emoji'] = emoji


df.head(10)

,text,label,repost,instant,fear,crowd,emoji
0,星巴克Starbucks詐騙邀請LINE訊息. 原始謠傳版本：. 統一星巴克starbuck...,1,0,0,0,0,0
1,转摘：\n我给大家透漏个家传秘密，每天出门前用棉签蘸点小磨香油，滴进两个鼻孔内，轻捏几下即可...,1,0,0,0,0,0
2,各位朋友請注意，剛才在承德路被警察攔下臨檢，因為當時沒有戴口罩，被警察詢問車上有沒有乘客，我...,1,0,0,0,1,0
3,亡國危機迫在眉睫- -家庭觀念及性別認同危機。這支影片看後你我都有責仼，請及時踴躍發聲！轉發...,1,1,0,0,0,0
4,(shiny)臭跩貓回來囉(shiny)\n限時活動發放貼圖主題\n超可愛白爛貓愛嗆人嗆不停...,1,0,0,0,0,0
5,有個重要的觀念，希望能再度提醒大家一下 ： 我們現在對付的是 “病毒”， 不是“細菌”。 ...,1,0,1,0,0,0
6,網路上流傳吃蕨菜會致癌真的假的,1,0,0,0,0,0
7,高雄鐵路地下化通車啟用……\n搭車贈一卡通紀念卡,1,0,0,0,0,0
8,这是我刚刚收到的消息：\n转清华群的消息：来自武汉抗击疫情第一线的消息，李留树（呼吸肿瘤博士...,1,0,0,0,0,1
9,准備一只圓珠筆，去芯，去芯，去芯。進電梯的時候拔下筆帽來，用筆尖去按按鈕，用完了馬上蓋上筆蓋...,1,0,0,0,0,0


In [119]:
features_1=df[['repost']]
features_2=df[['instant']]
features_3=df[['fear']]
features_4=df[['crowd']]
features_5=df[['emoji']]
labels=df['label']

features_all = pd.concat([features_1,
                          features_2,
                          features_3,
                          features_4,
                          features_5],
                         axis=1)  # check, axis=1 表示「橫向合併」

features_all.head(10)

,repost,instant,fear,crowd,emoji
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,0,0,0,0,0
5,0,1,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,1
9,0,0,0,0,0




*    random_state 確保訓練集與測試集的劃分方式固定。
*    test_size = 20%， 也就是說training_size = 80%。



In [120]:
feat_1_train, feat_1_test, feat_2_train, feat_2_test, feat_3_train, feat_3_test, feat_4_train, feat_4_test, feat_5_train, feat_5_test, label_train, label_test = train_test_split(
    features_1, features_2, features_3, features_4, features_5, labels, test_size=0.2, random_state=42)

In [121]:
print(feat_1_train, feat_1_test)

     repost
29        0
535       0
695       1
557       0
836       0
..      ...
106       0
270       0
860       0
435       0
102       1

[800 rows x 1 columns]      repost
521       0
737       0
740       0
660       0
411       0
..      ...
408       0
332       0
208       0
613       0
78        0

[200 rows x 1 columns]




*   為什麼要用value, (e.g.feat_1_train.**values**)

    因為Pandas DataFrame 無法直接轉換成 PyTorch Tensor，但 NumPy 陣列可以。





*   Torch.cat : 在相同的維度內拼接
*   Torch.stack(dim=n) : 會增加一個dim 在 第 n 維度



In [122]:
# Converting features into tensors
def convert_tensor(x):
    return torch.tensor(x, dtype=torch.float32)

feat_1_train_tensor = convert_tensor(feat_1_train.values)
feat_1_test_tensor = convert_tensor(feat_1_test.values)
feat_2_train_tensor = convert_tensor(feat_2_train.values)
feat_2_test_tensor = convert_tensor(feat_2_test.values)
feat_3_train_tensor = convert_tensor(feat_3_train.values)
feat_3_test_tensor = convert_tensor(feat_3_test.values)
feat_4_train_tensor = convert_tensor(feat_4_train.values)
feat_4_test_tensor = convert_tensor(feat_4_test.values)
feat_5_train_tensor = convert_tensor(feat_5_train.values)
feat_5_test_tensor = convert_tensor(feat_5_test.values)

feat_all_train_tensor = torch.cat([feat_1_train_tensor,
                                     feat_2_train_tensor,
                                     feat_3_train_tensor,
                                     feat_4_train_tensor,
                                     feat_5_train_tensor],
                                     dim=1)  # 在特徵維度合併

feat_all_test_tensor = torch.cat([feat_1_test_tensor,
                                    feat_2_test_tensor,
                                    feat_3_test_tensor,
                                    feat_4_test_tensor,
                                    feat_5_test_tensor],
                                    dim=1)

# Converting labels into tensors
label_train_tensor = torch.tensor(label_train.values, dtype=torch.long)
label_test_tensor = torch.tensor(label_test.values, dtype=torch.long)

# Inspecting the result
print(f"feat_train_tensor shape: {feat_all_train_tensor.shape}")
print(f"label_train_tensor shape: {label_train_tensor.shape}")
print(feat_all_train_tensor[:10])
print(label_train_tensor[:10])

print(f"feat_test_tensor shape: {feat_all_test_tensor.shape}")
print(f"label_test_tensor shape: {label_test_tensor.shape}")
print(feat_all_test_tensor[:10])
print(label_test_tensor[:10])

feat_train_tensor shape: torch.Size([800, 5])
label_train_tensor shape: torch.Size([800])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 0])
feat_test_tensor shape: torch.Size([200, 5])
label_test_tensor shape: torch.Size([200])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 1])


In [123]:
num_features = feat_all_train_tensor.shape[1]  # number of features
print('Number of features:', num_features)
num_classes = len(torch.unique(label_train_tensor))  # number of labels
print('Number of labels:',num_classes)

Number of features: 5
Number of labels: 2


In [129]:
class MultinomialNB:
    def __init__(self, num_classes, num_features, alpha=1.0):
        """
        Multinomial Naive Bayes model.
        Parameters:
            num_classes (int): Number of class labels.
            num_features (int): Number of features.
            alpha (float): Laplace smoothing parameter. Set alpha=0 to disable smoothing.
        """
        self.num_classes = num_classes
        self.num_features = num_features
        self.alpha = alpha  # Laplace smoothing
        self.class_log_prior_ = None  # log(P(C))
        self.feature_log_prob_ = None  # log(P(X|C))

    def compute_probabilities(self, X, y):
        """
        Compute class prior probabilities P(C) and feature probabilities P(X | C).
        Parameters:
            X (Tensor): (num_samples, num_features), Feature counts.
            y (Tensor): (num_samples,), Class labels.
        """
        # calculate the prior probability P(C)
        class_counts = torch.bincount(y, minlength=self.num_classes).float()
        class_prior = class_counts / class_counts.sum()  # P(C)
        self.class_log_prior_ = torch.log(class_prior)  # log(P(C))

        # printing out the prior probability
        for c in range(self.num_classes):
            print(f"Prior P(C={c}): {class_prior[c]:.4f}")

        # caluculating the conditional probability P(X | C)
        feature_counts = torch.zeros(self.num_classes, self.num_features)
        for c in range(self.num_classes):
            feature_counts[c] = X[y == c].sum(dim=0)  # 計算該類別的特徵計數

        # Avoiding division by zero problems when `alpha=0` via setting epsilon
        eps = 1e-10
        feature_prob = (feature_counts + self.alpha) / (feature_counts.sum(dim=1, keepdim=True) + self.alpha * self.num_features + eps)
        self.feature_log_prob_ = torch.log(feature_prob)  # log(P(X | C))


    def compute_log_likelihood(self, X):
        """
        Compute log likelihood for each sample.
        Parameters:
            X (Tensor): (num_samples, num_features), Feature counts.
        Returns:
            log_prob (Tensor): (num_samples, num_classes), Log probabilities.
        """
        return self.class_log_prior_ + X @ self.feature_log_prob_.T  # log P(C) + Σ log P(X | C)

In [145]:
print(f"feat_dev_fold shape: {feat_dev_fold.shape}")  # (num_samples, ?)
print(f"feature_log_prob_ shape: {model.feature_log_prob_.shape}")  # (num_classes, num_features)
feat_all_train_tensor.shape

feat_dev_fold shape: torch.Size([80, 1])
feature_log_prob_ shape: torch.Size([2, 5])


torch.Size([800, 5])

In [151]:
# defining 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# collecting the weighted F1-scores of each fold
f1_scores_weighted = []

# conducting 10-Fold Cross-Validation
for fold, (train_idx, dev_idx) in enumerate(kf.split(feat_train_tensor)):
    print(f"\nFold {fold+1}/10")

    # splitting data into the trainig set and the dev set
    feat_train_fold = feat_all_train_tensor[torch.tensor(train_idx)] # 要改成([800,5])
    feat_dev_fold = feat_all_train_tensor[torch.tensor(dev_idx)] # 這邊再想一下
    label_train_fold = label_train_tensor[torch.tensor(train_idx)]
    label_dev_fold = label_train_tensor[torch.tensor(dev_idx)]

    # It helps debug to put print() in your codes.
    print(f"feat_dev_fold shape: {feat_dev_fold.shape}")
    print(f"label_dev_fold shape: {label_dev_fold.shape}")

    # training
    model = MultinomialNB(num_classes, num_features, alpha=1.0)
    model.compute_probabilities(feat_train_fold, label_train_fold)

    # calculating log likelihood
    log_likihood = model.compute_log_likelihood(feat_dev_fold)

    # predicting based on log likihood
    label_dev_pred = log_likihood.argmax(dim=1)

    # It helps debug to put print() in your codes.
    print(f"label_dev_pred shape: {label_dev_pred.shape}")
    print(f"label_dev_fold shape: {label_dev_fold.shape}")

    # ensuring `label_dev_pred` and `label_dev_fold` have consistent shapes
    label_dev_pred_np = label_dev_pred.view(-1).numpy()
    label_dev_fold_np = label_dev_fold.view(-1).numpy()

    # calculating F1-score
    f1_weighted = f1_score(label_dev_fold_np, label_dev_pred_np, average='weighted')
    # weighted是改F-1 score的，可以自己改。

    # collecting weighted F1-scores of each fold
    f1_scores_weighted.append(f1_weighted)

    # printing out the weighted F1-score of each fold
    print(f"F1-Score (weighted): {f1_weighted:.4f}")

# calcualting the mean and the standard variation of weighted F1-scores
print("\n=== Cross-Validation Results ===")
print(f"F1-Score (weighted): Mean = {np.mean(f1_scores_weighted):.4f}, Std = {np.std(f1_scores_weighted):.4f}")


Fold 1/10
feat_dev_fold shape: torch.Size([80, 5])
label_dev_fold shape: torch.Size([80])
Prior P(C=0): 0.4958
Prior P(C=1): 0.5042
label_dev_pred shape: torch.Size([80])
label_dev_fold shape: torch.Size([80])
F1-Score (weighted): 0.4902

Fold 2/10
feat_dev_fold shape: torch.Size([80, 5])
label_dev_fold shape: torch.Size([80])
Prior P(C=0): 0.4972
Prior P(C=1): 0.5028
label_dev_pred shape: torch.Size([80])
label_dev_fold shape: torch.Size([80])
F1-Score (weighted): 0.4405

Fold 3/10
feat_dev_fold shape: torch.Size([80, 5])
label_dev_fold shape: torch.Size([80])
Prior P(C=0): 0.5028
Prior P(C=1): 0.4972
label_dev_pred shape: torch.Size([80])
label_dev_fold shape: torch.Size([80])
F1-Score (weighted): 0.3627

Fold 4/10
feat_dev_fold shape: torch.Size([80, 5])
label_dev_fold shape: torch.Size([80])
Prior P(C=0): 0.4875
Prior P(C=1): 0.5125
label_dev_pred shape: torch.Size([80])
label_dev_fold shape: torch.Size([80])
F1-Score (weighted): 0.3061

Fold 5/10
feat_dev_fold shape: torch.Size([

In [149]:
label_pred = model.compute_log_likelihood(feat_all_test_tensor).argmax(dim=1)
label_true = label_test_tensor

# Converting tensors into numpy format
label_pred = label_pred.numpy()
label_true = label_true.numpy()

In [150]:
accuracy_sklearn = accuracy_score(label_true, label_pred)
precision_sklearn = precision_score(label_true, label_pred, average='weighted')
recall_sklearn = recall_score(label_true, label_pred, average='weighted')
f1_sklearn = f1_score(label_true, label_pred, average='weighted')

# printind out the results
print("\n=== Evaluation Metrics (Scikit-learn) ===")
print(f"Accuracy : {accuracy_sklearn:.4f}")
print(f"Precision: {precision_sklearn:.4f}")
print(f"Recall   : {recall_sklearn:.4f}")
print(f"F1 Score : {f1_sklearn:.4f}")


=== Evaluation Metrics (Scikit-learn) ===
Accuracy : 0.4750
Precision: 0.4744
Recall   : 0.4750
F1 Score : 0.3741
